In [1]:
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer

In [2]:
table_3 = pd.read_csv('../Output_folder/3_Table.csv')
table_3

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,longitude,stars_y,review_count,is_open,attributes,categories,hours,label,pre_process,word_count_vector
0,7Lsue1__5v3-Y_PF2CS7bA,4LbxeQIyej3mr6HYYTsd7Q,SZU9c8V2GuREDN5KgyHFJw,1,0,0,0,First off the bathroom smell like dirty seafoo...,2016-04-23 22:06:09,Santa Barbara Shellfish Company,...,-119.685019,4.0,2404,1,"{'OutdoorSeating': 'True', 'RestaurantsAttire'...","Live/Raw Food, Restaurants, Seafood, Beer Bar,...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",0,first bathroom smell like dirty seafood almost...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,NZzgLC06h1lXpgqYeHVSxQ,ZbDR27cCgShAdKU2TQXwKg,q6661I3CGW0UB740E5Zjfw,1,1,0,0,"Quite possibly the worst ""indian"" food I have ...",2015-03-05 04:49:10,India House,...,-119.694577,2.0,136,0,"{'RestaurantsAttire': ""u'casual'"", 'WiFi': ""u'...","Home & Garden, Home Decor, Restaurants, Shoppi...","{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:...",0,quite possibly worst indian food ever experien...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,GLJgz8OT1Naw8CtisNTdow,PoTB7fwV--PaX-z_yhLXdA,5hJZe_v9Ax5GjXJCMQ9ijw,1,0,0,0,Last time I will use FedEx Office. Went in for...,2017-03-15 17:49:29,FedEx Office Print & Ship Center,...,-119.702806,2.0,53,1,{'BusinessAcceptsCreditCards': 'True'},"Printing Services, Education, Signmaking, Pres...","{'Monday': '7:30-21:0', 'Tuesday': '7:30-21:0'...",0,last time use fedex office went passport photo...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,IYleeBcfoXlgwMeRcEFAUQ,jYyUWgcy2FiEIMPwj988PQ,HQ-C47_Xi5it1KzwEc0u0A,1,0,0,0,Waited over 1 hour for pizza. And they called ...,2015-08-09 09:14:13,Barbarians,...,-119.696040,3.0,96,0,"{'Alcohol': ""u'beer_and_wine'"", 'RestaurantsRe...","Pizza, Bars, Sports Bars, Restaurants, Nightlife","{'Monday': '17:0-2:30', 'Tuesday': '17:0-2:30'...",0,waited hour pizza called number u employee rud...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,k0xQrwbjdb9DqoWSPmcG1g,HtPSa9IIR4q9bSSAcrbyIw,noByYNtDLQAra9ccqxdfDw,1,1,0,1,The staff of this H&M are some of the WORST re...,2015-09-12 23:46:16,H&M,...,-119.700460,3.0,24,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Women's Clothing, Accessories, Children's Clot...","{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",0,staff h worst retail employee ever encountered...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,SBxB-mPGBMBGQbW4pyNasA,GXn4ZsasLKh0qZ5g3nIqcQ,29YqJwOGEuAWqlHZxMc1OA,5,0,0,0,You would think that because I live in the Mis...,2005-09-23 18:58:51,Paradise Cafe,...,-119.696851,3.5,290,0,"{'BusinessParking': ""{'garage': True, 'street'...","American (Traditional), American (New), Nightl...","{'Monday': '11:0-15:0', 'Tuesday': '11:0-21:0'...",1,would think live mission district san francisc...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1984,YPBresB5TNp6366t03BbZg,Rw0vCeikk9aS2Z_CzXedFA,-ujBP1Dw0j1-Ffaz97-LXQ,5,0,0,0,Suuuuuperrrr knowledgeable staff - enjoyed Bri...,2018-08-14 03:55:18,Lama Dog Tap Room,...,-119.688480,4.0,238,1,"{'RestaurantsReservations': 'False', 'WiFi': ""...","Beer, Wine & Spirits, Pubs, Arts & Entertainme...","{'Monday': '0:0-0:0', 'Tuesday': '11:30-17:0',...",1,suuuuuperrrr knowledgeable staff enjoyed brigi...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1985,EaMgr7-58PVyqAQXsQn-KQ,PGAEw4DI5eZhLn-WfPPh-w,_W9lITxMHNInaS2Lv2XoUw,5,0,0,0,My husband and I came to Santa Barbara for our...,2017-08-09 04:31:56,Casa Del Mar Inn,...,-119.693248,4.0,194,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Hotels & Travel, Event Planning & Services, Ho...","{'Monday': '7:0-22:0', 'Tuesday': '7:0-22:0', ...",1,husband came santa barbara rd wedding annivers...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1986,tEWk3Ghwls4o_HC0L4JtlQ,eqek57im8IjOAhre5bUJPw,SZU9c8V2GuREDN5KgyHFJw,4,0,0,0,Sister and I split the loc

In [3]:
table_3.dtypes

review_id             object
user_id               object
business_id           object
stars_x                int64
useful                 int64
funny                  int64
cool                   int64
text                  object
date                  object
name                  object
address               object
city                  object
state                 object
postal_code            int64
latitude             float64
longitude            float64
stars_y              float64
review_count           int64
is_open                int64
attributes            object
categories            object
hours                 object
label                  int64
pre_process           object
word_count_vector     object
dtype: object

In [4]:
table_4 = table_3.drop(['user_id','useful','funny','cool', 'date','attributes', 'categories', 'hours'], axis=1)
table_4

,review_id,business_id,stars_x,text,name,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,label,pre_process,word_count_vector
0,7Lsue1__5v3-Y_PF2CS7bA,SZU9c8V2GuREDN5KgyHFJw,1,First off the bathroom smell like dirty seafoo...,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,0,first bathroom smell like dirty seafood almost...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,NZzgLC06h1lXpgqYeHVSxQ,q6661I3CGW0UB740E5Zjfw,1,"Quite possibly the worst ""indian"" food I have ...",India House,418 State St,Santa Barbara,CA,93101,34.416540,-119.694577,2.0,136,0,0,quite possibly worst indian food ever experien...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,GLJgz8OT1Naw8CtisNTdow,5hJZe_v9Ax5GjXJCMQ9ijw,1,Last time I will use FedEx Office. Went in for...,FedEx Office Print & Ship Center,1030 State St,Santa Barbara,CA,93101,34.422313,-119.702806,2.0,53,1,0,last time use fedex office went passport photo...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,IYleeBcfoXlgwMeRcEFAUQ,HQ-C47_Xi5it1KzwEc0u0A,1,Waited over 1 hour for pizza. And they called ...,Barbarians,511 State St,Santa Barbara,CA,93101,34.417006,-119.696040,3.0,96,0,0,waited hour pizza called number u employee rud...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,k0xQrwbjdb9DqoWSPmcG1g,noByYNtDLQAra9ccqxdfDw,1,The staff of this H&M are some of the WORST re...,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,1,0,staff h worst retail employee ever encountered...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,SBxB-mPGBMBGQbW4pyNasA,29YqJwOGEuAWqlHZxMc1OA,5,You would think that because I live in the Mis...,Paradise Cafe,702 Anacapa St,Santa Barbara,CA,93101,34.420035,-119.696851,3.5,290,0,1,would think live mission district san francisc...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1984,YPBresB5TNp6366t03BbZg,-ujBP1Dw0j1-Ffaz97-LXQ,5,Suuuuuperrrr knowledgeable staff - enjoyed Bri...,Lama Dog Tap Room,116 Santa Barbara St,Santa Barbara,CA,93101,34.415747,-119.688480,4.0,238,1,1,suuuuuperrrr knowledgeable staff enjoyed brigi...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1985,EaMgr7-58PVyqAQXsQn-KQ,_W9lITxMHNInaS2Lv2XoUw,5,My husband and I came to Santa Barbara for our...,Casa Del Mar Inn,18 Bath St,Santa Barbara,CA,93101,34.409619,-119.693248,4.0,194,1,1,husband came santa barbara rd wedding annivers...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1986,tEWk3Ghwls4o_HC0L4JtlQ,SZU9c8V2GuREDN5KgyHFJw,4,Sister and I split the local rock crab which i...,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,1,sister split local rock crab lb deliciousness ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
data = pd.read_csv('../Output_folder/Three_Table.csv')

In [6]:
data

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours,label,pre_process
0,7Lsue1__5v3-Y_PF2CS7bA,4LbxeQIyej3mr6HYYTsd7Q,SZU9c8V2GuREDN5KgyHFJw,1,0,0,0,First off the bathroom smell like dirty seafoo...,2016-04-23 22:06:09,Santa Barbara Shellfish Company,...,34.408715,-119.685019,4.0,2404,1,"{'OutdoorSeating': 'True', 'RestaurantsAttire'...","Live/Raw Food, Restaurants, Seafood, Beer Bar,...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",0,first bathroom smell like dirty seafood almost...
1,NZzgLC06h1lXpgqYeHVSxQ,ZbDR27cCgShAdKU2TQXwKg,q6661I3CGW0UB740E5Zjfw,1,1,0,0,"Quite possibly the worst ""indian"" food I have ...",2015-03-05 04:49:10,India House,...,34.416540,-119.694577,2.0,136,0,"{'RestaurantsAttire': ""u'casual'"", 'WiFi': ""u'...","Home & Garden, Home Decor, Restaurants, Shoppi...","{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:...",0,quite possibly worst indian food ever experien...
2,GLJgz8OT1Naw8CtisNTdow,PoTB7fwV--PaX-z_yhLXdA,5hJZe_v9Ax5GjXJCMQ9ijw,1,0,0,0,Last time I will use FedEx Office. Went in for...,2017-03-15 17:49:29,FedEx Office Print & Ship Center,...,34.422313,-119.702806,2.0,53,1,{'BusinessAcceptsCreditCards': 'True'},"Printing Services, Education, Signmaking, Pres...","{'Monday': '7:30-21:0', 'Tuesday': '7:30-21:0'...",0,last time use fedex office went passport photo...
3,IYleeBcfoXlgwMeRcEFAUQ,jYyUWgcy2FiEIMPwj988PQ,HQ-C47_Xi5it1KzwEc0u0A,1,0,0,0,Waited over 1 hour for pizza. And they called ...,2015-08-09 09:14:13,Barbarians,...,34.417006,-119.696040,3.0,96,0,"{'Alcohol': ""u'beer_and_wine'"", 'RestaurantsRe...","Pizza, Bars, Sports Bars, Restaurants, Nightlife","{'Monday': '17:0-2:30', 'Tuesday': '17:0-2:30'...",0,waited hour pizza called number u employee rud...
4,k0xQrwbjdb9DqoWSPmcG1g,HtPSa9IIR4q9bSSAcrbyIw,noByYNtDLQAra9ccqxdfDw,1,1,0,1,The staff of this H&M are some of the WORST re...,2015-09-12 23:46:16,H&M,...,34.420209,-119.700460,3.0,24,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Women's Clothing, Accessories, Children's Clot...","{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",0,staff h worst retail employee ever encountered...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,SBxB-mPGBMBGQbW4pyNasA,GXn4ZsasLKh0qZ5g3nIqcQ,29YqJwOGEuAWqlHZxMc1OA,5,0,0,0,You would think that because I live in the Mis...,2005-09-23 18:58:51,Paradise Cafe,...,34.420035,-119.696851,3.5,290,0,"{'BusinessParking': ""{'garage': True, 'street'...","American (Traditional), American (New), Nightl...","{'Monday': '11:0-15:0', 'Tuesday': '11:0-21:0'...",1,would think live mission district san francisc...
1984,YPBresB5TNp6366t03BbZg,Rw0vCeikk9aS2Z_CzXedFA,-ujBP1Dw0j1-Ffaz97-LXQ,5,0,0,0,Suuuuuperrrr knowledgeable staff - enjoyed Bri...,2018-08-14 03:55:18,Lama Dog Tap Room,...,34.415747,-119.688480,4.0,238,1,"{'RestaurantsReservations': 'False', 'WiFi': ""...","Beer, Wine & Spirits, Pubs, Arts & Entertainme...","{'Monday': '0:0-0:0', 'Tuesday': '11:30-17:0',...",1,suuuuuperrrr knowledgeable staff enjoyed brigi...
1985,EaMgr7-58PVyqAQXsQn-KQ,PGAEw4DI5eZhLn-WfPPh-w,_W9lITxMHNInaS2Lv2XoUw,5,0,0,0,My husband and I came to Santa Barbara for our...,2017-08-09 04:31:56,Casa Del Mar Inn,...,34.409619,-119.693248,4.0,194,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Hotels & Travel, Event Planning & Services, Ho...","{'Monday': '7:0-22:0', 'Tuesday': '7:0-22:0', ...",1,husband came santa barbara rd wedding annivers...
1986,tEWk3Ghwls4o_HC0L4JtlQ,eqek57im8IjOAhre5bUJPw,SZU9c8V2GuREDN5KgyHFJw,4,0,0,0,Sister and I split the local rock crab which i...,2014-01-04 03:53:14,Santa Barbara Shellfish Company,...,34.408715,-119.685019,4.0,2404,1,"{'OutdoorSeating': 'True', 'RestaurantsAttire'...","Live/Raw Food, Restaurants, Seafood, Beer Bar,...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",1,sister split local rock crab lb deliciousness ...


In [7]:
data.dtypes

review_id        object
user_id          object
business_id      object
stars_x           int64
useful            int64
funny             int64
cool              int64
text             object
date             object
name             object
address          object
city             object
state            object
postal_code       int64
latitude        float64
longitude       float64
stars_y         float64
review_count      int64
is_open           int64
attributes       object
categories       object
hours            object
label             int64
pre_process      object
dtype: object

In [8]:
new_data = data.drop(['user_id','useful','funny','cool', 'date','attributes', 'categories', 'hours'], axis=1)
new_data

,review_id,business_id,stars_x,text,name,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,label,pre_process
0,7Lsue1__5v3-Y_PF2CS7bA,SZU9c8V2GuREDN5KgyHFJw,1,First off the bathroom smell like dirty seafoo...,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,0,first bathroom smell like dirty seafood almost...
1,NZzgLC06h1lXpgqYeHVSxQ,q6661I3CGW0UB740E5Zjfw,1,"Quite possibly the worst ""indian"" food I have ...",India House,418 State St,Santa Barbara,CA,93101,34.416540,-119.694577,2.0,136,0,0,quite possibly worst indian food ever experien...
2,GLJgz8OT1Naw8CtisNTdow,5hJZe_v9Ax5GjXJCMQ9ijw,1,Last time I will use FedEx Office. Went in for...,FedEx Office Print & Ship Center,1030 State St,Santa Barbara,CA,93101,34.422313,-119.702806,2.0,53,1,0,last time use fedex office went passport photo...
3,IYleeBcfoXlgwMeRcEFAUQ,HQ-C47_Xi5it1KzwEc0u0A,1,Waited over 1 hour for pizza. And they called ...,Barbarians,511 State St,Santa Barbara,CA,93101,34.417006,-119.696040,3.0,96,0,0,waited hour pizza called number u employee rud...
4,k0xQrwbjdb9DqoWSPmcG1g,noByYNtDLQAra9ccqxdfDw,1,The staff of this H&M are some of the WORST re...,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,1,0,staff h worst retail employee ever encountered...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,SBxB-mPGBMBGQbW4pyNasA,29YqJwOGEuAWqlHZxMc1OA,5,You would think that because I live in the Mis...,Paradise Cafe,702 Anacapa St,Santa Barbara,CA,93101,34.420035,-119.696851,3.5,290,0,1,would think live mission district san francisc...
1984,YPBresB5TNp6366t03BbZg,-ujBP1Dw0j1-Ffaz97-LXQ,5,Suuuuuperrrr knowledgeable staff - enjoyed Bri...,Lama Dog Tap Room,116 Santa Barbara St,Santa Barbara,CA,93101,34.415747,-119.688480,4.0,238,1,1,suuuuuperrrr knowledgeable staff enjoyed brigi...
1985,EaMgr7-58PVyqAQXsQn-KQ,_W9lITxMHNInaS2Lv2XoUw,5,My husband and I came to Santa Barbara for our...,Casa Del Mar Inn,18 Bath St,Santa Barbara,CA,93101,34.409619,-119.693248,4.0,194,1,1,husband came santa barbara rd wedding annivers...
1986,tEWk3Ghwls4o_HC0L4JtlQ,SZU9c8V2GuREDN5KgyHFJw,4,Sister and I split the local rock crab which i...,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,1,sister split local rock crab lb deliciousness ...


In [9]:
merged_df = table_4.merge(new_data, how = 'left', on = ['review_id'])
merged_df

,review_id,business_id_x,stars_x_x,text_x,name_x,address_x,city_x,state_x,postal_code_x,latitude_x,...,city_y,state_y,postal_code_y,latitude_y,longitude_y,stars_y_y,review_count_y,is_open_y,label_y,pre_process_y
0,7Lsue1__5v3-Y_PF2CS7bA,SZU9c8V2GuREDN5KgyHFJw,1,First off the bathroom smell like dirty seafoo...,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,...,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,0,first bathroom smell like dirty seafood almost...
1,NZzgLC06h1lXpgqYeHVSxQ,q6661I3CGW0UB740E5Zjfw,1,"Quite possibly the worst ""indian"" food I have ...",India House,418 State St,Santa Barbara,CA,93101,34.416540,...,Santa Barbara,CA,93101,34.416540,-119.694577,2.0,136,0,0,quite possibly worst indian food ever experien...
2,GLJgz8OT1Naw8CtisNTdow,5hJZe_v9Ax5GjXJCMQ9ijw,1,Last time I will use FedEx Office. Went in for...,FedEx Office Print & Ship Center,1030 State St,Santa Barbara,CA,93101,34.422313,...,Santa Barbara,CA,93101,34.422313,-119.702806,2.0,53,1,0,last time use fedex office went passport photo...
3,IYleeBcfoXlgwMeRcEFAUQ,HQ-C47_Xi5it1KzwEc0u0A,1,Waited over 1 hour for pizza. And they called ...,Barbarians,511 State St,Santa Barbara,CA,93101,34.417006,...,Santa Barbara,CA,93101,34.417006,-119.696040,3.0,96,0,0,waited hour pizza called number u employee rud...
4,k0xQrwbjdb9DqoWSPmcG1g,noByYNtDLQAra9ccqxdfDw,1,The staff of this H&M are some of the WORST re...,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,...,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,1,0,staff h worst retail employee ever encountered...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,SBxB-mPGBMBGQbW4pyNasA,29YqJwOGEuAWqlHZxMc1OA,5,You would think that because I live in the Mis...,Paradise Cafe,702 Anacapa St,Santa Barbara,CA,93101,34.420035,...,Santa Barbara,CA,93101,34.420035,-119.696851,3.5,290,0,1,would think live mission district san francisc...
1984,YPBresB5TNp6366t03BbZg,-ujBP1Dw0j1-Ffaz97-LXQ,5,Suuuuuperrrr knowledgeable staff - enjoyed Bri...,Lama Dog Tap Room,116 Santa Barbara St,Santa Barbara,CA,93101,34.415747,...,Santa Barbara,CA,93101,34.415747,-119.688480,4.0,238,1,1,suuuuuperrrr knowledgeable staff enjoyed brigi...
1985,EaMgr7-58PVyqAQXsQn-KQ,_W9lITxMHNInaS2Lv2XoUw,5,My husband and I came to Santa Barbara for our...,Casa Del Mar Inn,18 Bath St,Santa Barbara,CA,93101,34.409619,...,Santa Barbara,CA,93101,34.409619,-119.693248,4.0,194,1,1,husband came santa barbara rd wedding annivers...
1986,tEWk3Ghwls4o_HC0L4JtlQ,SZU9c8V2GuREDN5KgyHFJw,4,Sister and I split the local rock crab which i...,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,...,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,1,sister split local rock crab lb deliciousness ...


In [10]:
drop_merged = merged_df.dropna()
drop_merged

,review_id,business_id_x,stars_x_x,text_x,name_x,address_x,city_x,state_x,postal_code_x,latitude_x,...,city_y,state_y,postal_code_y,latitude_y,longitude_y,stars_y_y,review_count_y,is_open_y,label_y,pre_process_y
0,7Lsue1__5v3-Y_PF2CS7bA,SZU9c8V2GuREDN5KgyHFJw,1,First off the bathroom smell like dirty seafoo...,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,...,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,0,first bathroom smell like dirty seafood almost...
1,NZzgLC06h1lXpgqYeHVSxQ,q6661I3CGW0UB740E5Zjfw,1,"Quite possibly the worst ""indian"" food I have ...",India House,418 State St,Santa Barbara,CA,93101,34.416540,...,Santa Barbara,CA,93101,34.416540,-119.694577,2.0,136,0,0,quite possibly worst indian food ever experien...
2,GLJgz8OT1Naw8CtisNTdow,5hJZe_v9Ax5GjXJCMQ9ijw,1,Last time I will use FedEx Office. Went in for...,FedEx Office Print & Ship Center,1030 State St,Santa Barbara,CA,93101,34.422313,...,Santa Barbara,CA,93101,34.422313,-119.702806,2.0,53,1,0,last time use fedex office went passport photo...
3,IYleeBcfoXlgwMeRcEFAUQ,HQ-C47_Xi5it1KzwEc0u0A,1,Waited over 1 hour for pizza. And they called ...,Barbarians,511 State St,Santa Barbara,CA,93101,34.417006,...,Santa Barbara,CA,93101,34.417006,-119.696040,3.0,96,0,0,waited hour pizza called number u employee rud...
4,k0xQrwbjdb9DqoWSPmcG1g,noByYNtDLQAra9ccqxdfDw,1,The staff of this H&M are some of the WORST re...,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,...,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,1,0,staff h worst retail employee ever encountered...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,SBxB-mPGBMBGQbW4pyNasA,29YqJwOGEuAWqlHZxMc1OA,5,You would think that because I live in the Mis...,Paradise Cafe,702 Anacapa St,Santa Barbara,CA,93101,34.420035,...,Santa Barbara,CA,93101,34.420035,-119.696851,3.5,290,0,1,would think live mission district san francisc...
1984,YPBresB5TNp6366t03BbZg,-ujBP1Dw0j1-Ffaz97-LXQ,5,Suuuuuperrrr knowledgeable staff - enjoyed Bri...,Lama Dog Tap Room,116 Santa Barbara St,Santa Barbara,CA,93101,34.415747,...,Santa Barbara,CA,93101,34.415747,-119.688480,4.0,238,1,1,suuuuuperrrr knowledgeable staff enjoyed brigi...
1985,EaMgr7-58PVyqAQXsQn-KQ,_W9lITxMHNInaS2Lv2XoUw,5,My husband and I came to Santa Barbara for our...,Casa Del Mar Inn,18 Bath St,Santa Barbara,CA,93101,34.409619,...,Santa Barbara,CA,93101,34.409619,-119.693248,4.0,194,1,1,husband came santa barbara rd wedding annivers...
1986,tEWk3Ghwls4o_HC0L4JtlQ,SZU9c8V2GuREDN5KgyHFJw,4,Sister and I split the local rock crab which i...,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,...,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,1,sister split local rock crab lb deliciousness ...


In [11]:
drop_merged.dtypes

review_id             object
business_id_x         object
stars_x_x              int64
text_x                object
name_x                object
address_x             object
city_x                object
state_x               object
postal_code_x          int64
latitude_x           float64
longitude_x          float64
stars_y_x            float64
review_count_x         int64
is_open_x              int64
label_x                int64
pre_process_x         object
word_count_vector     object
business_id_y         object
stars_x_y              int64
text_y                object
name_y                object
address_y             object
city_y                object
state_y               object
postal_code_y          int64
latitude_y           float64
longitude_y          float64
stars_y_y            float64
review_count_y         int64
is_open_y              int64
label_y                int64
pre_process_y         object
dtype: object

In [12]:
clean_merge = drop_merged.drop(['text_x', 'word_count_vector', 'business_id_y', 'stars_x_y', 'text_y', 'name_y', 'address_y', 'city_y', 'state_y', 'postal_code_y', 'latitude_y', 'longitude_y', 'stars_y_y', 'review_count_y', 'is_open_y', 'label_y', 'pre_process_y' ], axis=1)
clean_merge

,review_id,business_id_x,stars_x_x,name_x,address_x,city_x,state_x,postal_code_x,latitude_x,longitude_x,stars_y_x,review_count_x,is_open_x,label_x,pre_process_x
0,7Lsue1__5v3-Y_PF2CS7bA,SZU9c8V2GuREDN5KgyHFJw,1,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,0,first bathroom smell like dirty seafood almost...
1,NZzgLC06h1lXpgqYeHVSxQ,q6661I3CGW0UB740E5Zjfw,1,India House,418 State St,Santa Barbara,CA,93101,34.416540,-119.694577,2.0,136,0,0,quite possibly worst indian food ever experien...
2,GLJgz8OT1Naw8CtisNTdow,5hJZe_v9Ax5GjXJCMQ9ijw,1,FedEx Office Print & Ship Center,1030 State St,Santa Barbara,CA,93101,34.422313,-119.702806,2.0,53,1,0,last time use fedex office went passport photo...
3,IYleeBcfoXlgwMeRcEFAUQ,HQ-C47_Xi5it1KzwEc0u0A,1,Barbarians,511 State St,Santa Barbara,CA,93101,34.417006,-119.696040,3.0,96,0,0,waited hour pizza called number u employee rud...
4,k0xQrwbjdb9DqoWSPmcG1g,noByYNtDLQAra9ccqxdfDw,1,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,1,0,staff h worst retail employee ever encountered...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,SBxB-mPGBMBGQbW4pyNasA,29YqJwOGEuAWqlHZxMc1OA,5,Paradise Cafe,702 Anacapa St,Santa Barbara,CA,93101,34.420035,-119.696851,3.5,290,0,1,would think live mission district san francisc...
1984,YPBresB5TNp6366t03BbZg,-ujBP1Dw0j1-Ffaz97-LXQ,5,Lama Dog Tap Room,116 Santa Barbara St,Santa Barbara,CA,93101,34.415747,-119.688480,4.0,238,1,1,suuuuuperrrr knowledgeable staff enjoyed brigi...
1985,EaMgr7-58PVyqAQXsQn-KQ,_W9lITxMHNInaS2Lv2XoUw,5,Casa Del Mar Inn,18 Bath St,Santa Barbara,CA,93101,34.409619,-119.693248,4.0,194,1,1,husband came santa barbara rd wedding annivers...
1986,tEWk3Ghwls4o_HC0L4JtlQ,SZU9c8V2GuREDN5KgyHFJw,4,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,1,sister split local rock crab lb deliciousness ...


In [13]:
clean_merge.dtypes

review_id          object
business_id_x      object
stars_x_x           int64
name_x             object
address_x          object
city_x             object
state_x            object
postal_code_x       int64
latitude_x        float64
longitude_x       float64
stars_y_x         float64
review_count_x      int64
is_open_x           int64
label_x             int64
pre_process_x      object
dtype: object

In [14]:
#this cell changes the column names
label_merge = clean_merge.rename(columns={"business_id_x": "business_id", 
                    "stars_x_x": "stars", 
                    "name_x": "name", 
                    "address_x": "address",
                    "city_x": "city",
                    "state_x": "state",
                    "postal_code_x": "postal_code",
                    "latitude_x": "latitude",
                    "longitude_x": "longitude",
                    "stars_y_x": "review_stars",
                    "review_count_x": "review_count",
                    "is_open_x": "is_open",
                     "label_x": "label",
                     "pre_process_x": "pre_process"})

In [15]:
label_merge

,review_id,business_id,stars,name,address,city,state,postal_code,latitude,longitude,review_stars,review_count,is_open,label,pre_process
0,7Lsue1__5v3-Y_PF2CS7bA,SZU9c8V2GuREDN5KgyHFJw,1,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,0,first bathroom smell like dirty seafood almost...
1,NZzgLC06h1lXpgqYeHVSxQ,q6661I3CGW0UB740E5Zjfw,1,India House,418 State St,Santa Barbara,CA,93101,34.416540,-119.694577,2.0,136,0,0,quite possibly worst indian food ever experien...
2,GLJgz8OT1Naw8CtisNTdow,5hJZe_v9Ax5GjXJCMQ9ijw,1,FedEx Office Print & Ship Center,1030 State St,Santa Barbara,CA,93101,34.422313,-119.702806,2.0,53,1,0,last time use fedex office went passport photo...
3,IYleeBcfoXlgwMeRcEFAUQ,HQ-C47_Xi5it1KzwEc0u0A,1,Barbarians,511 State St,Santa Barbara,CA,93101,34.417006,-119.696040,3.0,96,0,0,waited hour pizza called number u employee rud...
4,k0xQrwbjdb9DqoWSPmcG1g,noByYNtDLQAra9ccqxdfDw,1,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,1,0,staff h worst retail employee ever encountered...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,SBxB-mPGBMBGQbW4pyNasA,29YqJwOGEuAWqlHZxMc1OA,5,Paradise Cafe,702 Anacapa St,Santa Barbara,CA,93101,34.420035,-119.696851,3.5,290,0,1,would think live mission district san francisc...
1984,YPBresB5TNp6366t03BbZg,-ujBP1Dw0j1-Ffaz97-LXQ,5,Lama Dog Tap Room,116 Santa Barbara St,Santa Barbara,CA,93101,34.415747,-119.688480,4.0,238,1,1,suuuuuperrrr knowledgeable staff enjoyed brigi...
1985,EaMgr7-58PVyqAQXsQn-KQ,_W9lITxMHNInaS2Lv2XoUw,5,Casa Del Mar Inn,18 Bath St,Santa Barbara,CA,93101,34.409619,-119.693248,4.0,194,1,1,husband came santa barbara rd wedding annivers...
1986,tEWk3Ghwls4o_HC0L4JtlQ,SZU9c8V2GuREDN5KgyHFJw,4,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,1,sister split local rock crab lb deliciousness ...


In [16]:
label_merge['label'] = label_merge['label'].astype(str)

In [17]:
label_merge

,review_id,business_id,stars,name,address,city,state,postal_code,latitude,longitude,review_stars,review_count,is_open,label,pre_process
0,7Lsue1__5v3-Y_PF2CS7bA,SZU9c8V2GuREDN5KgyHFJw,1,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,0,first bathroom smell like dirty seafood almost...
1,NZzgLC06h1lXpgqYeHVSxQ,q6661I3CGW0UB740E5Zjfw,1,India House,418 State St,Santa Barbara,CA,93101,34.416540,-119.694577,2.0,136,0,0,quite possibly worst indian food ever experien...
2,GLJgz8OT1Naw8CtisNTdow,5hJZe_v9Ax5GjXJCMQ9ijw,1,FedEx Office Print & Ship Center,1030 State St,Santa Barbara,CA,93101,34.422313,-119.702806,2.0,53,1,0,last time use fedex office went passport photo...
3,IYleeBcfoXlgwMeRcEFAUQ,HQ-C47_Xi5it1KzwEc0u0A,1,Barbarians,511 State St,Santa Barbara,CA,93101,34.417006,-119.696040,3.0,96,0,0,waited hour pizza called number u employee rud...
4,k0xQrwbjdb9DqoWSPmcG1g,noByYNtDLQAra9ccqxdfDw,1,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,1,0,staff h worst retail employee ever encountered...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,SBxB-mPGBMBGQbW4pyNasA,29YqJwOGEuAWqlHZxMc1OA,5,Paradise Cafe,702 Anacapa St,Santa Barbara,CA,93101,34.420035,-119.696851,3.5,290,0,1,would think live mission district san francisc...
1984,YPBresB5TNp6366t03BbZg,-ujBP1Dw0j1-Ffaz97-LXQ,5,Lama Dog Tap Room,116 Santa Barbara St,Santa Barbara,CA,93101,34.415747,-119.688480,4.0,238,1,1,suuuuuperrrr knowledgeable staff enjoyed brigi...
1985,EaMgr7-58PVyqAQXsQn-KQ,_W9lITxMHNInaS2Lv2XoUw,5,Casa Del Mar Inn,18 Bath St,Santa Barbara,CA,93101,34.409619,-119.693248,4.0,194,1,1,husband came santa barbara rd wedding annivers...
1986,tEWk3Ghwls4o_HC0L4JtlQ,SZU9c8V2GuREDN5KgyHFJw,4,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,1,sister split local rock crab lb deliciousness ...


In [18]:
label_merge.dtypes

review_id        object
business_id      object
stars             int64
name             object
address          object
city             object
state            object
postal_code       int64
latitude        float64
longitude       float64
review_stars    float64
review_count      int64
is_open           int64
label            object
pre_process      object
dtype: object

In [19]:
label_merge['label'] = label_merge['label'].replace(['0'],'Negative')
label_merge['label'] = label_merge['label'].replace(['1'],'Positive')

In [24]:
nu_label_merge = label_merge
nu_label_merge

,review_id,business_id,stars,name,address,city,state,postal_code,latitude,longitude,review_stars,review_count,is_open,label,pre_process
0,7Lsue1__5v3-Y_PF2CS7bA,SZU9c8V2GuREDN5KgyHFJw,1,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,Negative,first bathroom smell like dirty seafood almost...
1,NZzgLC06h1lXpgqYeHVSxQ,q6661I3CGW0UB740E5Zjfw,1,India House,418 State St,Santa Barbara,CA,93101,34.416540,-119.694577,2.0,136,0,Negative,quite possibly worst indian food ever experien...
2,GLJgz8OT1Naw8CtisNTdow,5hJZe_v9Ax5GjXJCMQ9ijw,1,FedEx Office Print & Ship Center,1030 State St,Santa Barbara,CA,93101,34.422313,-119.702806,2.0,53,1,Negative,last time use fedex office went passport photo...
3,IYleeBcfoXlgwMeRcEFAUQ,HQ-C47_Xi5it1KzwEc0u0A,1,Barbarians,511 State St,Santa Barbara,CA,93101,34.417006,-119.696040,3.0,96,0,Negative,waited hour pizza called number u employee rud...
4,k0xQrwbjdb9DqoWSPmcG1g,noByYNtDLQAra9ccqxdfDw,1,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,1,Negative,staff h worst retail employee ever encountered...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,SBxB-mPGBMBGQbW4pyNasA,29YqJwOGEuAWqlHZxMc1OA,5,Paradise Cafe,702 Anacapa St,Santa Barbara,CA,93101,34.420035,-119.696851,3.5,290,0,Positive,would think live mission district san francisc...
1984,YPBresB5TNp6366t03BbZg,-ujBP1Dw0j1-Ffaz97-LXQ,5,Lama Dog Tap Room,116 Santa Barbara St,Santa Barbara,CA,93101,34.415747,-119.688480,4.0,238,1,Positive,suuuuuperrrr knowledgeable staff enjoyed brigi...
1985,EaMgr7-58PVyqAQXsQn-KQ,_W9lITxMHNInaS2Lv2XoUw,5,Casa Del Mar Inn,18 Bath St,Santa Barbara,CA,93101,34.409619,-119.693248,4.0,194,1,Positive,husband came santa barbara rd wedding annivers...
1986,tEWk3Ghwls4o_HC0L4JtlQ,SZU9c8V2GuREDN5KgyHFJw,4,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,Positive,sister split local rock crab lb deliciousness ...


In [34]:
nu_label_merge['pre_process'] = nu_label_merge['pre_process'].str.split()

In [35]:
nu_label_merge

,review_id,business_id,stars,name,address,city,state,postal_code,latitude,longitude,review_stars,review_count,is_open,label,pre_process
0,7Lsue1__5v3-Y_PF2CS7bA,SZU9c8V2GuREDN5KgyHFJw,1,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,Negative,"[first, bathroom, smell, like, dirty, seafood,..."
1,NZzgLC06h1lXpgqYeHVSxQ,q6661I3CGW0UB740E5Zjfw,1,India House,418 State St,Santa Barbara,CA,93101,34.416540,-119.694577,2.0,136,0,Negative,"[quite, possibly, worst, indian, food, ever, e..."
2,GLJgz8OT1Naw8CtisNTdow,5hJZe_v9Ax5GjXJCMQ9ijw,1,FedEx Office Print & Ship Center,1030 State St,Santa Barbara,CA,93101,34.422313,-119.702806,2.0,53,1,Negative,"[last, time, use, fedex, office, went, passpor..."
3,IYleeBcfoXlgwMeRcEFAUQ,HQ-C47_Xi5it1KzwEc0u0A,1,Barbarians,511 State St,Santa Barbara,CA,93101,34.417006,-119.696040,3.0,96,0,Negative,"[waited, hour, pizza, called, number, u, emplo..."
4,k0xQrwbjdb9DqoWSPmcG1g,noByYNtDLQAra9ccqxdfDw,1,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,1,Negative,"[staff, h, worst, retail, employee, ever, enco..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,SBxB-mPGBMBGQbW4pyNasA,29YqJwOGEuAWqlHZxMc1OA,5,Paradise Cafe,702 Anacapa St,Santa Barbara,CA,93101,34.420035,-119.696851,3.5,290,0,Positive,"[would, think, live, mission, district, san, f..."
1984,YPBresB5TNp6366t03BbZg,-ujBP1Dw0j1-Ffaz97-LXQ,5,Lama Dog Tap Room,116 Santa Barbara St,Santa Barbara,CA,93101,34.415747,-119.688480,4.0,238,1,Positive,"[suuuuuperrrr, knowledgeable, staff, enjoyed, ..."
1985,EaMgr7-58PVyqAQXsQn-KQ,_W9lITxMHNInaS2Lv2XoUw,5,Casa Del Mar Inn,18 Bath St,Santa Barbara,CA,93101,34.409619,-119.693248,4.0,194,1,Positive,"[husband, came, santa, barbara, rd, wedding, a..."
1986,tEWk3Ghwls4o_HC0L4JtlQ,SZU9c8V2GuREDN5KgyHFJw,4,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,Positive,"[sister, split, local, rock, crab, lb, delicio..."


In [36]:
squeaky_merge = nu_label_merge

In [37]:
squeaky_merge.to_excel('../Output_Folder/squeaky_merge.xlsx')

In [38]:
squeaky_merge.to_csv('../Output_Folder/squeaky_merge.csv')